In [33]:
import asammdf
import hashlib
import get_files
import pony.orm
from pony.orm.core import EntityMeta
from pony import orm

pony.orm.set_sql_debug(False)

In [16]:
db = pony.orm.Database()
db.bind(
    provider='sqlite',
    filename=':memory:',
)

class MDF(db.Entity):
    name = pony.orm.Required(
        str,
    )
    version = pony.orm.Required(
        str,
    )
    sha256 = pony.orm.Required(
        str, 
        unique=True,
    )
    channels = pony.orm.Set(
        'Channel',
    )
    
class Channel(db.Entity):
    name = pony.orm.Required(
        str,
        unique=True,
    )
    mdfs = pony.orm.Set(
        "MDF",
    )
db.generate_mapping(create_tables=True)

In [23]:
def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: 
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), "{cls} is not a database entity".format(cls=cls)

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj

In [24]:
data_files = get_files.get_files(
    directory="../Data/",
    extensions=[".mdf", ".mf4"],
)

In [25]:
for data_file in data_files:
    break

In [26]:
# def index_data_file(data_file):
with open(data_file, "rb") as fid:
    sha256 = hashlib.sha256(fid.read()).hexdigest()

mdf = asammdf.MDF(data_file)

channels = list()
mdf.channels_db.keys()
for channel in mdf.channels_db.keys():
    channel_ = upsert(Channel, {"name": channel})
    channels.append(channel_)

In [32]:
M_ = upsert(
    cls=MDF,
    get={"sha256": sha256},
    set={
        "name": data_file,
        "version": mdf.version,
        "channels": channels,
    },
)
M_

MDF[1]

In [31]:
db.commit()